<a href="https://colab.research.google.com/github/NaohiroTawara/B3_seminor2018/blob/master/B3_seminor2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colaboratory（Google Colab）とは

・ Google Colabは、完全にクラウドで実行される Jupyter ノートブック環境です．

・感覚的には，クラウド上のバーチャル環境を割り当てられ，無料で使えるイメージです

## 注意事項



### ９０分ルール，１２時間ルール

 
*   計算途中でこのページを離れても計算は継続されますが，ページから離れたまま９０分が経過すると計算が強制的に止まります．そのため９０分以上かかる計算を行う場合はブラウザを閉じないようにしてください

*   ページから離れるかにかかわらず，起動時から１２時間が経過するとすべての環境がリセットされます．そのため１２時間以上計算がかかる場合はこのスクリプトは適用できません．




## 使い方

挿入ー＞コードセルを選択するとコードを打ち込み，実行可能なセルが追加されます.

コードセルはpython のコマンドの他に，!マークを先頭につけることで仮想環境のbashコマンドを実行することができます

コマンド入力後にセルの左にある▶を押すとコードが実行されます


In [0]:
a=1
b=2
print("a + b = {}".format(a+b))

In [0]:
!ls # カレントディレクトリ内の一覧を見るコマンド

In [0]:
!cat /etc/lsb-release #Linux のバージョンを調べるコマンド

In [0]:
!free -h # メモリを調べるコマンド

In [0]:
!cat /proc/cpuinfo | grep -e "model name" -e "processor" #cpuの型番を調べるコマンド

## GPUの割当

ランタイムー＞ランタイムのタイプを変更

ハードウェアアクセラレータをGPUに変更

#matplotlibによる可視化の例

In [0]:
import matplotlib.pyplot as plt
import numpy as np

x = np.arange(20)
y = [x_i + np.random.randn(1) for x_i in x]
a, b = np.polyfit(x, y, 1)
_ = plt.plot(x, y, 'o', np.arange(20), a*np.arange(20)+b, '-')

# chainer によるDNN構築

## chainer のインストール
・chainerはデフォルトでは google colabで使えないので，仮想環境にインストールする必要があります 

In [0]:
!apt -y install libcusparse8.0 libnvrtc8.0 libnvtoolsext1
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
!pip install https://github.com/kmaehashi/chainer-colab/releases/download/2018-02-06/cupy_cuda80-4.0.0b3-cp36-cp36m-linux_x86_64.whl
!pip install 'chainer==4.0.0b3'
!apt-get install graphviz
!pip install 'chaineripy'

In [3]:
import chainer
#gpu が使えるかチェック
print('GPU availability:', chainer.cuda.available)
print('cuDNN availablility:', chainer.cuda.cudnn_enabled)

GPU availability: True
cuDNN availablility: True


## MNIST数字文字認識

### 学習データの用意

In [0]:
from chainer.datasets import mnist
# set matplotlib so that we can see our drawing inside this notebook
%matplotlib inline
import matplotlib.pyplot as plt

# Download the MNIST data if you haven't downloaded it yet
train, test = mnist.get_mnist(withlabel=True, ndim=1)
# Display an example from the MNIST dataset.
# `x` contains the input image array and `t` contains that target class
# label as an integer.
plt.figure()
for i in range(1,11):
  x, t = train[i]
  plt.subplot(2,5,i)
  plt.imshow(x.reshape(28, 28), cmap='gray')
  plt.title('label: {}'.format(t))

### 必要なパッケージのインポート

In [0]:
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training
from chainer.training import extensions
from chainer.serializers import save_npz, load_npz

### ネットワーク構造の定義

In [0]:
# Network definition
class MLP(chainer.Chain):

    def __init__(self, n_units, n_out):
        super(MLP, self).__init__()
        with self.init_scope():
            # the size of the inputs to each layer will be inferred
            self.l1 = L.Linear(None, n_units)  # n_in -> n_units
            self.l2 = L.Linear(None, n_units)  # n_units -> n_units
            self.l3 = L.Linear(None, n_out)  # n_units -> n_out

    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)

### ネットワークの学習

In [0]:
gpu = 0 #gpu id (CPUを使う場合は-1)
unit = 100 # 中間層のノード数
batchsize =128 # ミニバッチサイズ
epoch =20 # エポック数
out = 'results' # 結果の出力先
frequency=-1 # 学習経過の出力タイミング -1の場合エポックごと
    
# Set up a neural network to train
# Classifier reports softmax cross entropy loss and accuracy at every
# iteration, which will be used by the PrintReport extension below.
mlp = MLP(unit, 10)
model = L.Classifier(mlp)
if gpu >= 0:
  # Make a specified GPU current
  chainer.cuda.get_device_from_id(gpu).use()
  model.to_gpu()  # GPU にコピー

# 最適化法の選択
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

# MNIST datasetの取得
train, test = chainer.datasets.get_mnist()

train_iter = chainer.iterators.SerialIterator(train, batchsize)
test_iter = chainer.iterators.SerialIterator(test, batchsize,
                                               repeat=False, shuffle=False)

# Set up a trainer
updater = training.StandardUpdater(train_iter, optimizer, device=gpu)
trainer = training.Trainer(updater, (epoch, 'epoch'), out=out)

# Evaluate the model with the test dataset for each epoch
trainer.extend(extensions.Evaluator(test_iter, model, device=gpu))

# Dump a computational graph from 'loss' variable at the first iteration
# The "main" refers to the target link of the "main" optimizer.
trainer.extend(extensions.dump_graph('main/loss'))

# Take a snapshot for each specified epoch
frequency = epoch if frequency == -1 else max(1, frequency)
trainer.extend(extensions.snapshot(), trigger=(frequency, 'epoch'))

# Write a log of evaluation statistics for each epoch
trainer.extend(extensions.LogReport())

# Save two plot images to the result dir
if extensions.PlotReport.available():
  trainer.extend(
      extensions.PlotReport(['main/loss', 'validation/main/loss'],
                            'epoch', file_name='loss.png'))
  trainer.extend(
      extensions.PlotReport(
          ['main/accuracy', 'validation/main/accuracy'],
          'epoch', file_name='accuracy.png'))

trainer.extend(extensions.PrintReport(
    ['epoch', 'main/loss', 'validation/main/loss',
     'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))

# Print a progress bar to stdout
#trainer.extend(extensions.ProgressBar())

# Run the training
trainer.run()

# Save the trained model
save_npz(out+'/model', mlp)

### 学習したモデルを用いてテストデータを識別してみる

In [0]:
# Create the infrence (evaluation) model as the previous model
infer_model = MLP(unit, 10)

# Load the saved parameters into the parameters of the new inference model to overwrite 
load_npz(out+'/model', infer_model)

# Send the model to utilize GPU by to_GPU
if gpu >= 0:
    infer_model.to_gpu(gpu)

In [0]:
# Get a test image and label
x, t = test[0]
plt.imshow(x.reshape(28, 28), cmap='gray')
plt.show()
print('label:', t)

In [32]:
# change the shape to minibutch. 
# In this example, the size of minibatch is 1. 
# Inference using any mini-batch size can be performed.

print(x.shape, end=' -> ')
x = x[None, ...]
print(x.shape)

# to calculate by GPU, send the data to GPU, too. 
if gpu >= 0:
    x = chainer.cuda.to_gpu(x, gpu)

# forward calculation of the model by sending X
y = infer_model(x)

# The result is given as Variable, then we can take a look at the contents by the attribute, .data. 
y = y.data

# send the gpu result to cpu
y = chainer.cuda.to_cpu(y)

# The most probable number by looking at the argmax
pred_label = y.argmax(axis=1)

print('predicted label:', pred_label[0])

(1, 1, 784) -> (1, 1, 1, 784)
predicted label: 7


### より詳細な結果の解析

#### モデルの読み込み

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np

_, test = chainer.datasets.get_mnist()
x,t = test._datasets

filename=out+'/model' 
model = MLP(unit, 10)
if gpu >= 0:
    # Make a specified GPU current
    chainer.cuda.get_device_from_id(gpu).use()
    model.to_gpu()  # Copy the model to the GPU
load_npz(filename, model)

x_dnn=np.empty([0, 10])
for i in range(0,len(x),batchsize):
    x_batch = x[i:i+batchsize]
    if gpu >= 0:
        x_batch = chainer.Variable(chainer.cuda.to_gpu(x_batch))
    x_dnn = np.r_[x_dnn, chainer.cuda.to_cpu(model(x_batch).data)] #model(x_batch).to_cpu().data]


#### 学習曲線の表示

In [0]:
from IPython.display import Image
Image(out+'/accuracy.png')

In [0]:
from IPython.display import Image
Image(out+'/loss.png')

####ニューラルネットの構造を表示

In [0]:
import pydotplus
from IPython.display import Image
graph = pydotplus.graphviz.graph_from_dot_file(out+'/cg.dot')
graph.write_png(out+'/cg.png')
Image(graph.create_png())

#### 元の画像をtSNEで二次元に圧縮し表示する

・ tSNE はデータ間の距離が保存しつつ次元を削減する次元圧縮法

・ この例では画像(28×28次元)を2次元に圧縮して散布図で表示する
（各プロットが１枚の画像を表し，色がクラス（数字番号）を示す）



In [0]:
x_tsn= TSNE(n_components=2).fit_transform(x[:1000])
plt.scatter(x_tsn[:1000,0], x_tsn[:1000,1],c=t[:1000], alpha=0.5, cmap='tab20c')
plt.title('original image 784 (=28 x 28) dim.  -> 2 dim')
plt.show()

#### DNNの出力（10次元のベクトル）をtSNEで二次元に圧縮し表示する

In [0]:
x_tsn= TSNE(n_components=2).fit_transform(x_dnn[:1000])
plt.scatter(x_tsn[:1000,0], x_tsn[:1000,1],c=t[:1000], alpha=0.5, cmap='tab20c')
plt.title('DNN output 10 dim.  -> 2 dim')
plt.show()

# アニメ画像認識

[リンクテキスト](https://qiita.com/mitmul/items/5502ecdd2f0b444c427f) から引用

### データのダウンロード

In [0]:
%%bash
if [ ! -d animeface-character-dataset ]; then
    curl -L -O http://www.nurs.or.jp/~nagadomi/animeface-character-dataset/data/animeface-character-dataset.zip
    unzip animeface-character-dataset.zip
    rm -rf animeface-character-dataset.zip
fi

In [0]:
ls animeface-character-dataset/thumb

### データセットオブジェクトの作成

In [0]:
import os
import glob
from itertools import chain
from chainer.datasets import LabeledImageDataset


# 画像フォルダ
IMG_DIR = 'animeface-character-dataset/thumb'

# 各キャラクターごとのフォルダ
dnames = glob.glob('{}/*'.format(IMG_DIR))

# 画像ファイルパス一覧
fnames = [glob.glob('{}/*.png'.format(d)) for d in dnames
          if not os.path.exists('{}/ignore'.format(d))]
fnames = list(chain.from_iterable(fnames))

# それぞれにフォルダ名から一意なIDを付与
labels = [os.path.basename(os.path.dirname(fn)) for fn in fnames]
dnames = [os.path.basename(d) for d in dnames
          if not os.path.exists('{}/ignore'.format(d))]
labels = [dnames.index(l) for l in labels]


# データセット作成
d = LabeledImageDataset(list(zip(fnames, labels)))

TransformDatasetへの変換。これは、データセットオブジェクトと各データへの変換を表す関数を取るラッパークラスで、これを使うとdata augmentationや前処理などを行う部分をデータセットクラスの外に用意しておくことができる。

In [0]:
from chainer.datasets import TransformDataset
from PIL import Image

width, height = 160, 160

# 画像のresize関数
def resize(img):
    img = Image.fromarray(img.transpose(1, 2, 0))
    img = img.resize((width, height), Image.BICUBIC)
    return np.asarray(img).transpose(2, 0, 1)

# 各データに行う変換
def transform(inputs):
    img, label = inputs
    img = img[:3, ...]
    img = resize(img.astype(np.uint8))
    img = img - mean[:, None, None]
    img = img.astype(np.float32)
    # ランダムに左右反転
    if np.random.rand() > 0.5:
        img = img[..., ::-1]
    return img, label

# 変換付きデータセットにする
td = TransformDataset(d, transform)

平均画像の計算

In [0]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm_notebook

# 平均画像が未計算なら計算する
if not os.path.exists('image_mean.npy'):
    # 変換をかまさないバージョンの学習用データセットで平均を計算したい
    t, _ = datasets.split_dataset_random(d, int(len(d) * 0.8), seed=0)

    mean = np.zeros((3, height, width))
    for img, _ in tqdm_notebook(t, desc='Calc mean'):
        img = resize(img[:3].astype(np.uint8))
        mean += img
    mean = mean / float(len(d))
    np.save('image_mean', mean)
else:
    mean = np.load('image_mean.npy')
    
mean = mean.mean(axis=(1, 2))


学習用と，検証用データセットに分割．全体の８０％を学習に，残りの２０％を検証用にする

In [0]:
from chainer import datasets

train, valid = datasets.split_dataset_random(td, int(len(d) * 0.8), seed=0)

### モデルの構築

####学習済みモデルのダウンロード

In [0]:
%%bash
if [ ! -f illust2vec_ver200.caffemodel ]; then
    curl -L -O https://github.com/rezoo/illustration2vec/releases/download/v2.0.0/illust2vec_ver200.caffemodel
fi

In [0]:
%%bash
pip install Pillow
pip install tqdm

#### モデルの定義

In [0]:
import dill

import chainer
import chainer.links as L
import chainer.functions as F

from chainer import Chain
from chainer.links.caffe import CaffeFunction
from chainer import serializers

class Illust2Vec(Chain):

    CAFFEMODEL_FN = 'illust2vec_ver200.caffemodel'

    def __init__(self, n_classes, unchain=True):
        w = chainer.initializers.HeNormal()        
        model = CaffeFunction(self.CAFFEMODEL_FN)  # CaffeModelを読み込んで保存します。（時間がかかります）
        del model.encode1  # メモリ節約のため不要なレイヤを削除します。
        del model.encode2
        del model.forwards['encode1']
        del model.forwards['encode2']
        model.layers = model.layers[:-2]

        super(Illust2Vec, self).__init__()
        with self.init_scope():
            self.trunk = model  # 元のIllust2Vecモデルをtrunkとしてこのモデルに含めます。
            self.fc7 = L.Linear(None, 4096, initialW=w)
            self.bn7 = L.BatchNormalization(4096)
            self.fc8 = L.Linear(4096, n_classes, initialW=w)

    def __call__(self, x):
        h = self.trunk({'data': x}, ['conv6_3'])[0]  # 元のIllust2Vecモデルのconv6_3の出力を取り出します。
        h.unchain_backward()
        h = F.dropout(F.relu(self.bn7(self.fc7(h))))  # ここ以降は新しく追加した層です。
        return self.fc8(h)

n_classes = len(dnames)
model = Illust2Vec(n_classes)
model = L.Classifier(model)

### 学習のスタート

In [0]:
batchsize = 64
gpu_id = 0
initial_lr = 0.01
lr_drop_epoch = 10
lr_drop_ratio = 0.1
train_epoch = 20

if gpu_id >= 0:
  # Make a specified GPU current
  chainer.cuda.get_device_from_id(gpu_id).use()
  model.to_gpu() 
  
train_iter = iterators.MultiprocessIterator(train, batchsize)
valid_iter = iterators.MultiprocessIterator(
    valid, batchsize, repeat=False, shuffle=False)

optimizer = optimizers.MomentumSGD(lr=initial_lr)
optimizer.setup(model)
optimizer.add_hook(chainer.optimizer.WeightDecay(0.0001))

updater = training.StandardUpdater(
    train_iter, optimizer, device=gpu_id)

trainer = training.Trainer(updater, (train_epoch, 'epoch'), out='AnimeFace-result')
trainer.extend(extensions.LogReport())
trainer.extend(extensions.observe_lr())

# 標準出力に書き出したい値
trainer.extend(extensions.PrintReport(
    ['epoch',
     'main/loss',
     'main/accuracy',
     'val/main/loss',
     'val/main/accuracy',
     'elapsed_time',
     'lr']))

# ロスのプロットを毎エポック自動的に保存
trainer.extend(extensions.PlotReport(
        ['main/loss',
         'val/main/loss'],
        'epoch', file_name='loss.png'))

# 精度のプロットも毎エポック自動的に保存
trainer.extend(extensions.PlotReport(
        ['main/accuracy',
         'val/main/accuracy'],
        'epoch', file_name='accuracy.png'))

# モデルのtrainプロパティをFalseに設定してvalidationするextension
trainer.extend(extensions.Evaluator(valid_iter, model, device=gpu_id), name='val')

# 指定したエポックごとに学習率をlr_drop_ratio倍にする
trainer.extend(
    extensions.ExponentialShift('lr', lr_drop_ratio),
    trigger=(lr_drop_epoch, 'epoch'))

trainer.run()

### 結果の解析

#### 学習曲線の表示

In [0]:
from IPython.display import Image
Image(filename='AnimeFace-result/loss.png')

In [0]:
Image(filename='AnimeFace-result/accuracy.png')

#### 検証セットに対する識別結果の表示

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt

from PIL import Image
from chainer import cuda

chainer.config.train = False
for _ in range(10):
    x, t = valid[np.random.randint(len(valid))]
    x = cuda.to_gpu(x)
    y = F.softmax(model.predictor(x[None, ...]))

    pred = os.path.basename(dnames[int(y.data.argmax())])
    label = os.path.basename(dnames[t])

    print('pred:', pred, 'label:', label, pred == label)

    x = cuda.to_cpu(x)
    x += mean[:, None, None]
    x = x / 256
    x = np.clip(x, 0, 1)
    plt.imshow(x.transpose(1, 2, 0))
    plt.show()